In [1]:
# Whole dataset
! gdown 1atKYBZ9Pq3-zT0_BC_nZqMx0W-wjEW01 # all training data
! gdown 1FhT3m_ApKzX615JzshB5-d-j6S91-6oz # all validation data
! gdown 16p0td9GgJRb9AP8i4HlX-xZGI2u849uA # all testing data

# https://drive.google.com/file/d/1atKYBZ9Pq3-zT0_BC_nZqMx0W-wjEW01/view?usp=sharing

! mkdir open_llm
! mv train.jsonl valid.jsonl test.jsonl open_llm/

! pip install datasets

Downloading...
From (original): https://drive.google.com/uc?id=1atKYBZ9Pq3-zT0_BC_nZqMx0W-wjEW01
From (redirected): https://drive.google.com/uc?id=1atKYBZ9Pq3-zT0_BC_nZqMx0W-wjEW01&confirm=t&uuid=58b9be7f-1904-4a97-85fb-a3a10fcf3444
To: /content/train.jsonl
100% 292M/292M [00:07<00:00, 37.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FhT3m_ApKzX615JzshB5-d-j6S91-6oz
To: /content/valid.jsonl
100% 55.1M/55.1M [00:01<00:00, 53.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=16p0td9GgJRb9AP8i4HlX-xZGI2u849uA
To: /content/test.jsonl
100% 39.1M/39.1M [00:00<00:00, 51.5MB/s]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.8 MB/s eta 0:0

In [2]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from datasets import Dataset
import pandas as pd
import re

In [3]:
pretrained_model_name = "bert-base-uncased"
model = TFAutoModel.from_pretrained(pretrained_model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
df_train = pd.read_json("open_llm/train.jsonl", lines=True)
df_valid = pd.read_json("open_llm/valid.jsonl", lines=True)
df_test = pd.read_json("open_llm/test.jsonl", lines=True)

df_train = df_train.drop(columns=['extra', 'source'])
df_valid = df_valid.drop(columns=['extra', 'source'])
df_test = df_test.drop(columns=['extra', 'source'])

In [6]:
df_train

,uid,text,label
0,[urlsf_subset00]-[15],The dangers of Illinois as a ‘right to work’ s...,0
1,[urlsf_subset00]-[15],"The governor of Illinois, Gov. Rauner, has req...",1
2,[urlsf_subset00]-[83],Check current weather conditions\n\nIt’s going...,0
3,[urlsf_subset00]-[83],Check current weather conditions It’s going to...,1
4,[urlsf_subset00]-[89],"On Thursday, the president of the United State...",0
...,...,...,...
101279,[urlsf_subset05]-[389773],Halifax Water has imposed mandatory water cons...,1
101280,[urlsf_subset05]-[389814],"Less than a month into his tenure, Donald Trum...",0
101281,[urlsf_subset05]-[389814],"Less than a month into his tenure, Donald Trum...",1
101282,[urlsf_subset05]-[389832],Georgia antagonists love to cite a well-worn q...,0


In [7]:
def text_process(mess):
    """
    Process text to:
    1. Remove punctuation (including all Unicode quotes)
    2. Convert text to lowercase
    3. Return cleaned text without removing stopwords
    """
    # Remove all punctuation using regex
    mess = re.sub(r"[^\w\s]", "", mess)

    # Convert the text to lowercase
    mess = mess.lower()

    # Return the cleaned text
    return mess

In [8]:
df_train['text'] = df_train['text'].apply(text_process)
df_valid['text'] = df_valid['text'].apply(text_process)
df_test['text'] = df_test['text'].apply(text_process)

In [9]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=512)

In [10]:
# Convert pandas dataframes to Hugging Face datasets
hf_train = Dataset.from_pandas(df_train)
hf_valid = Dataset.from_pandas(df_valid)
hf_test = Dataset.from_pandas(df_test)

# Apply the tokenization
hf_train_encoded = hf_train.map(tokenize, batched=True)
hf_valid_encoded = hf_valid.map(tokenize, batched=True)
hf_test_encoded = hf_test.map(tokenize, batched=True)

# Preview the results
print(hf_train_encoded)
print(hf_valid_encoded)
print(hf_test_encoded)

Map:   0%|          | 0/101284 [00:00<?, ? examples/s]

Map:   0%|          | 0/19650 [00:00<?, ? examples/s]

Map:   0%|          | 0/13952 [00:00<?, ? examples/s]

Dataset({
    features: ['uid', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 101284
})
Dataset({
    features: ['uid', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 19650
})
Dataset({
    features: ['uid', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 13952
})


In [11]:
# setting 'input_ids', 'attention_mask', 'token_type_ids', and 'label'
# to the tensorflow format. Now if you access this dataset you will get these
# columns in `tf.Tensor` format

hf_train_encoded.set_format('tf',
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])
hf_valid_encoded.set_format(
    type='tf',
    columns=['input_ids', 'attention_mask', 'token_type_ids', 'label']
)

BATCH_SIZE = 32

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]

# converting train split of `emotions_encoded` to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(hf_train_encoded[:])
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# ... doing the same for test set ...
test_dataset = tf.data.Dataset.from_tensor_slices(hf_test_encoded[:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [12]:
valid_dataset = tf.data.Dataset.from_tensor_slices(hf_valid_encoded[:])
valid_dataset = valid_dataset.batch(BATCH_SIZE)
valid_dataset = valid_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [13]:
inp, out = next(iter(train_dataset)) # a batch from train_dataset
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
array([[  101,  3259, 21689, ...,  2013, 24581,   102],
       [  101,  1037,  5294, ...,     0,     0,     0],
       [  101,  3324,  6630, ...,     0,     0,     0],
       ...,
       [  101,  7890,  4817, ...,     0,     0,     0],
       [  101,  1996,  4245, ...,  9223,  6922,   102],
       [  101,  1996,  4245, ...,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0]])>} 

 tf.Tensor([0 1 0 1 0 1 0 1 0 1 0 1 

In [14]:
inp, out = next(iter(valid_dataset)) # a batch from train_dataset
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
array([[  101,  9317,  2258, ...,  2278,  2581,   102],
       [  101,  1037,  3232, ...,     0,     0,     0],
       [  101,  2624,  3799, ...,     0,     0,     0],
       ...,
       [  101,  3520, 14163, ...,     0,     0,     0],
       [  101,  2746,  2574, ...,     0,     0,     0],
       [  101,  2746,  2574, ...,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>} 

 tf.Tensor([0 1 0 1 0 1 0 1 0 1 0 1 

In [15]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, model, num_classes, dropout=0.3):
        super(BERTForClassification, self).__init__()
        self.bert = model
        self.dropout = tf.keras.layers.Dropout(dropout)
        self.classifier = tf.keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs, training=False):
        outputs = self.bert(**inputs)
        pooled_output = outputs[1]  # CLS token embedding
        x = self.dropout(pooled_output, training=training)
        return self.classifier(x)

In [16]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
classifier = BERTForClassification(model, num_classes=2, dropout=0.5)

classifier.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [17]:
history = classifier.fit(
    train_dataset,
    epochs=2,
    validation_data=valid_dataset
)

Epoch 1/2
3166/3166 [==============================] - 1456s 447ms/step - loss: 0.1914 - accuracy: 0.9196 - val_loss: 0.1922 - val_accuracy: 0.9306
Epoch 2/2
3166/3166 [==============================] - 1411s 446ms/step - loss: 0.0723 - accuracy: 0.9741 - val_loss: 0.0849 - val_accuracy: 0.9707


In [18]:
hf_test_encoded.set_format(
    type='tf',
    columns=['input_ids', 'attention_mask', 'token_type_ids', 'label']
)

# ... doing the same for test set ...
test_dataset = tf.data.Dataset.from_tensor_slices(hf_test_encoded[:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [19]:
# Make predictions on the test dataset
predictions = classifier.predict(test_dataset)

# Extract predicted labels and probabilities (if needed)
predicted_labels = predictions.argmax(axis=-1)

436/436 [==============================] - 66s 144ms/step


In [20]:
df_test['predicted_label'] = predicted_labels

df_test

,uid,text,label,predicted_label
0,[urlsf_subset09]-[13],media playback is unsupported on your device m...,0,0
1,[urlsf_subset09]-[13],officials have confirmed that the toxic red sl...,1,1
2,[urlsf_subset09]-[29],jazzman said i understand you cant ban sprawl ...,0,0
3,[urlsf_subset09]-[29],jazzman said i understand you cant ban sprawl ...,1,1
4,[urlsf_subset09]-[36],the egyptian parliament witnessed a new disput...,0,0
...,...,...,...,...
13947,[urlsf_subset09]-[389512],a letter from the federal bureau of alcohol to...,1,1
13948,[urlsf_subset09]-[389538],a us army black hawk helicopter is seen photo1...,0,0
13949,[urlsf_subset09]-[389538],a black hawk helicopter belonging to the us mi...,1,1
13950,[urlsf_subset09]-[389611],letter from an immigration official at heathro...,0,0


In [21]:
y_true = df_test['label']
y_pred = df_test['predicted_label']


In [22]:
# caculate f1 and accuracy
from sklearn.metrics import f1_score, accuracy_score

f1 = f1_score(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)

print(f1, accuracy)

0.9711416041872967 0.970756880733945


In [23]:
df_test_predictions = pd.read_json("open_llm/test.jsonl", lines=True)

df_test_predictions['predicted_label'] = predicted_labels

df_test_predictions

,uid,text,extra,source,label,predicted_label
0,[urlsf_subset09]-[13],Media playback is unsupported on your device M...,"{'source': 'openweb', 'variant': 'original'}",openweb,0,0
1,[urlsf_subset09]-[13],Officials have confirmed that the toxic red sl...,"{'source': 'chatgpt', 'variant': 'original'}",chatgpt,1,1
2,[urlsf_subset09]-[29],Jazzman said: I understand you can't ban spraw...,"{'source': 'openweb', 'variant': 'original'}",openweb,0,0
3,[urlsf_subset09]-[29],Jazzman said: I understand you can't ban spraw...,"{'variant': 'original', 'source': 'llama'}",llama,1,1
4,[urlsf_subset09]-[36],The Egyptian parliament witnessed a new disput...,"{'source': 'openweb', 'variant': 'original'}",openweb,0,0
...,...,...,...,...,...,...
13947,[urlsf_subset09]-[389512],"A letter from the federal Bureau of Alcohol, T...","{'variant': 'original', 'source': 'llama'}",llama,1,1
13948,[urlsf_subset09]-[389538],A U.S. Army Black Hawk helicopter is seen. (Ph...,"{'source': 'openweb', 'variant': 'original'}",openweb,0,0
13949,[urlsf_subset09]-[389538],A Black Hawk helicopter belonging to the U.S. ...,"{'source': 'chatgpt', 'variant': 'original'}",chatgpt,1,1
13950,[urlsf_subset09]-[389611],Letter from an immigration official at Heathro...,"{'source': 'openweb', 'variant': 'original'}",openweb,0,0


In [24]:
df_test_predictions.to_csv('test_result.csv', index=False)

In [25]:
# save classifier_weights.h5 to drive
save_path = "/content/drive/MyDrive/Thesis/Models/BERT/v2"

classifier.save_weights(save_path)
classifier.save_weights('classifier_weights.h5')

In [26]:
import os

# Path to save the classifier_weights.h5 file in Google Drive
drive_path = '/content/drive/MyDrive/Thesis/Models/BERT/'

# Ensure the folder exists
os.makedirs(drive_path, exist_ok=True)

# Define the path to the file you want to save
source_path = 'classifier_weights.h5'  # The current path of the file in Colab
destination_path = os.path.join(drive_path, 'classifier_weights_v2.h5')

# Save the file to Google Drive by simply copying it
with open(source_path, 'rb') as src_file:
    with open(destination_path, 'wb') as dst_file:
        dst_file.write(src_file.read())

# Confirm the file has been saved
print(f"Classifier weights saved to: {destination_path}")

Classifier weights saved to: /content/drive/MyDrive/Thesis/Models/BERT/classifier_weights_v2.h5


In [27]:
import os

# Define the directory where you want to save the model
save_path = "/content/drive/MyDrive/Thesis/Models/BERT"
os.makedirs(save_path, exist_ok=True)

# Save the entire model to the specified path
classifier.save(save_path)

print(f"Model saved to {save_path}")

import json

# Save optimizer configuration
optimizer_config = {
    'learning_rate': 1e-5
}

# Save the configuration to a JSON file
config_path = os.path.join(save_path, 'optimizer_config.json')
with open(config_path, 'w') as f:
    json.dump(optimizer_config, f)

print(f"Optimizer configuration saved to {config_path}")

# Save the training history
history_path = os.path.join(save_path, 'training_history.json')
with open(history_path, 'w') as f:
    json.dump(history.history, f)

print(f"Training history saved to {history_path}")

Model saved to /content/drive/MyDrive/Thesis/Models/BERT
Optimizer configuration saved to /content/drive/MyDrive/Thesis/Models/BERT/optimizer_config.json
Training history saved to /content/drive/MyDrive/Thesis/Models/BERT/training_history.json
